In [1]:
import numpy as np
import numpy.random as rn
import math as m
import matplotlib.pyplot as plt
from __future__ import division
import time

G = 6.67e-11
R_jup = 7.785e+11
M_jup = 1.898e+27
M_sun = 1.989e+30
V_jup = m.sqrt(G*M_sun/R_jup)
R_earth = 1.496e+11 #Smallest R for an asteroid

t_kep = 2 * m.pi * R_jup**1.5 / m.sqrt(G*M_sun) # keplerian period jupiter
t_year = 365 * 24 * 3600

In [2]:
nbodies = 50
t_end = 2 * t_kep # keplerian period jupiter
dt = 0.05 * t_year

bodieslist = ["Sun", "Jupiter"]
initcond = np.zeros((nbodies,7))
positions = {}
positions["Sun"] = np.array([0,0,0,0,0,0,M_sun]) #x,y,z,vx, vy, vz
positions["Jupiter"] = np.array([R_jup,0,0,0,V_jup,0,M_jup])

In [3]:
def f12(Body1,Body2,position_1):
    if Body1 == Body2:
        return np.array([0,0,0])
    pos = position_1-positions[Body2][0:3]
    dist = np.linalg.norm(pos)
    return -G* positions[Body2][-1]*pos/(dist)**3


def force(Body1, position_1):
    F = 0
    for k in ["Jupiter", "Sun"]:
        F = F + f12(Body1, k, position_1)
    return F


def asteroid_init():
    r_asteroids = np.linspace(R_earth,R_jup,n_bodies)
    v_kepler = np.sqrt(G*M_sun/r_asteroids)
    for r in r_asteroids:
        theta = 0       #Phase of the orbit
        v = m.sqrt(G*M_sun/r) * rn.uniform(1, 1.01)
        bodieslist.append("Body "+str(i+1))
        positions[bodieslist[i+2]] = np.array([r*m.cos(theta),r*m.sin(theta),0,v*m.cos(theta+m.pi/2),v*m.sin(theta+m.pi/2),0, 0])
        
    

In [4]:
def RK4_gravitational(Body, dt):
    
    k1 = np.concatenate((positions[Body][3:6], force(Body, positions[Body][:3]))) * dt
    k2 = np.concatenate((positions[Body][3:6] + k1[3:] * 0.5, force(Body, positions[Body][:3]  + k1[:3] * 0.5))) * dt
    k3 = np.concatenate((positions[Body][3:6] + k2[3:] * 0.5, force(Body, positions[Body][:3]  + k2[:3] * 0.5))) * dt
    k4 = np.concatenate((positions[Body][3:6] + k3[3:], force(Body, positions[Body][:3]  + k3[:3]))) * dt
    positions[Body][:6] = positions[Body][:6] + k1 / 6 + k2 / 3 + k3 / 3 + k4 / 6
    
    return positions[Body]


def EulerCromer_gravitational(Body, dt):
    
    v_np1 = positions[Body][3:6] + force(Body, positions[Body][:3]) * dt
    x_np1 = positions[Body][:3] + v_np1 * dt
    positions[Body][:6] = np.concatenate((x_np1, v_np1))
    
    return positions[Body]


def integrator(t_end, dt, int_type):
    asteroid_init() #comment this line if you want to test accuracy
#     fig, ax = plt.subplots(1,1, figsize=(8,6))
    for i in xrange(int(t_end/dt)):
        temporalbodielist = bodieslist
        for j, Body in enumerate(temporalbodielist):
            if int_type == 'RK4':
                positions[Body] = RK4_gravitational(Body, dt)
            else:
                positions[Body] = EulerCromer_gravitational(Body, dt)
            #Condition to get rid of asteroids flying away
            if positions[Body][0] > 3*R_jup or positions[Body][1] > 3* R_jup or positions[Body][0] < -3*R_jup or positions[Body][1] < -3* R_jup:  #Eliminating from the list asteroids scaping
                bodieslist.remove(Body)
        
        if i % 25 == 0:
            ax.plot(positions['Sun'][0] / R_jup, positions['Sun'][1] / R_jup, '.', c='y')
            ax.plot(positions['Jupiter'][0] / R_jup, positions['Jupiter'][1] / R_jup, '.', c='b')
    plt.show()
    return positions

time_1 = time.time()
integrator(t_end, dt, 'RK4')
print 'Computation time: ' + str(time.time() - time_1) + ' s'

NameError: global name 'n_bodies' is not defined